# Optimizer Backpropagation
Learning representations by back-propagating errors is a paper by [Rumelhart, Hinton, & Williams (1986)](https://www.nature.com/articles/323533a0), describing how NN may learn by repeatedly adjusting weights.

There are many algorithms that train neural networks, and each have their own benefits and consequences. Modern alogrithms are chosen for the ability to parallelize the vectors, for efficient computation on GPU shaders.

Fig 1: Method comparison, taken from [Sebastian Ruder](http://sebastianruder.com/optimizing-gradient-descent/index.html#visualizationofalgorithms).

![Training Techniques](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/contours_evaluation_optimizers.gif "Training Techniques")

## Classic backpropagation
[Backpropagation](https://en.wikipedia.org/wiki/Backpropagation) works by calculating the weight change $v_i$ for collective weight $\theta_i$ in the neural network:
$$
\theta_t = \theta_{t-1} - v_t
$$
for every iteration $t$. The weight change is calculated depending on the training algorithm. Classic backpropagation calculates a gradient for every weight in the neural network w.r.t. an error function $J$. The gradient is then scaled by a learning rate $\eta$:
$$
v_t = \eta \nabla_{\theta_{t-1}} J(\theta_{t-1}).
$$

The error function in classic backpropagation is the mean squared error (MSE)
$$
J(\theta_t) = \frac{1}{2N} \sum_{i=1}^N \left( \hat{y}_i - y_i \right)^2,
$$
in the $t^\text{th}$ layer, using the prediction $\hat{y}_i$ for expected $y_i$.

The learning rate $\eta$ is an important concept: setting the learning rate too low will converge to a good solution, but take many iterations. If the learning rate is too high, the process may jump out of the minima it is trying to descend into.

Backpropagation is also a type of gradient descent -- the gradient with respect to each weight provides an indication as how to modify the weight for better convergence. The gradient of the error function is implicit in the weight-value summation; we will now denote $w^k_{ij}$ for the weight between the $j^\text{th}$ neuron in the $k-1$ and the $i^\text{th}$ neuron in layer $k$, such that the bias $$
b_i^k = w_{0i}^k,
$$
and activation is now compactly expressed for fixed output $o_j^k$
$$
a_i^k = \sum_{j=0}^N{w_{ji}^k o_j^{k-1}}
$$
where $N$ is the number of neurons in the $k-1$ layer.

If we consider the MSE function above, we can express the derivative as
$$
\frac{\partial J(\theta)}{\partial w_{ij}^k} = \frac{1}{N}\sum_{d=1}^N \frac{\partial}{\partial w_{ij}^k} \left( \frac{1}{2}\left( \hat{y}_d - y_d \right)^2 \right) = \frac{1}{N} \sum_{d=1}^N \frac{\partial J_d}{\partial w_{ij}^k}
$$

Using a result derived in the backpropagation theory, we find that the derivative of the error function may be expressed
$$
\frac{\partial J}{\partial w_{ij}^k} = \frac{\partial E}{\partial a_j^k} o_i^{k-1},
$$
where the shorthand
$$
\delta_j^k := \frac{\partial E}{\partial a_j^k}
$$
is often used in the literature as the *error at layer $k$* due to *neuron $j$*.

### More detail
Additional good resources here on [Brilliant.org](https://brilliant.org/wiki/backpropagation/).

The overall NN may be expressed as a matrix multiplication
$$
g(x) := f^L(W^L f^{L-1}(W^{L-1}\cdots f^1(W^1 x) \cdots))
$$
where $f^l$ is the activation function at layer $l$, $W^l$ is the weights between layer $l-1$ and $l$. These weights are also expressed, as in previous sections, by the tensor $w^l_{jk}$ for the weight between the $k^\text{th}$ neuron in the $l-1$ and the $j^\text{th}$ neuron in layer $l$. 


The loss of the network is expressed by the two-argument function
$$
C = C(y_i, g(x_i))
$$
For a *fixed* input-output pair $(x_i, y_i)$, we vary the weights, and are interested in the gradient
$$
\frac{\partial C}{\partial w^l_{jk}}
$$
for backpropagation. However, performing this task for each neuron pair $j,k$ is inefficient -- backpropagation instead computes the gradient across a whole layer, specifically the gradient of the weighted input of each layer $\delta^l$, from the last layer to the first.

Let us denote the weighted input of each layer by $z^l$, and the output of layer $l$ by the *activation value* $a^l$. For backpropagation, the activation value and the derivatives of the activation function
$$
\left( f^l \right)^{\prime} = \left. \frac{\partial f^l }{\partial z^l} \right\vert_{z^l}
$$
must be cached.

The derivative of the loss in terms on inputs is expressed by chain-rule as total derivatives
$$
\frac{d C}{d a^L} \cdot \frac{d a^L}{d z^L} \cdot \frac{d z^L}{d a^{L-1}} \cdots \frac{da^1}{dz^1} \cdot \frac{\partial z^1}{\partial x},
$$
i.e. the derivatives of the activation functions and matrices of the weights:
$$
\frac{d C}{d a^L}\cdot \left( f^L \right)^{\prime} \cdot W^L \cdot \left( f^{L-1} \right)^{\prime} \cdot W^{L-1} \cdots \left( f^1 \right)^{\prime} \cdot W^1.
$$

The total gradient $\nabla$ is the transpose of the above, describing the derivative of the output in terms of the input:
$$
\nabla_x C = \left( W^1 \right)^T \cdot \left( f^1 \right)^{\prime} \cdots \left( W^{L-1} \right)^T \cdot \left( f^{L-1} \right)^{\prime} \cdot \left( W^L \right)^T \cdot \left( f^L \right)^{\prime} \cdot \nabla_{a^L} C.
$$

Backpropagation is then evaluating this expression from layer $L$ to layer $1$. There exists a slight catch; since the gradient isn't a compelte expression, we have an extra multiplication. For ease of notation, we introduce the auxiliary quantity $\delta^l$ for the parial product, interpreted as the *error at level $l$* and is a vector with dimension equal to the number of neurons at layer $l$,
$$
\delta^l := \left( f^l \right)^{\prime} \cdot \left( W^{l+1} \right)^T \cdots \left( W^{L-1} \right)^T \cdot \left( f^{L-1} \right)^{\prime} \cdot \left( W^L \right)^T \cdot \left( f^L \right)^{\prime} \cdot \nabla_{a^L} C.
$$

The gradient of the weight in layer $l$ is consequently
$$
\nabla_{W^l} C = \delta^l \left( a^{l-1} \right)^T
$$

We can easily compute the error recursively as
$$
\delta^{l-1} = \left( f^{l-1} \right)^{\prime} \cdot \left( W^l \right)^T \cdot \delta^l,
$$
starting from the desired output in the final layer.

## Momentum backpropagation
Momentum backpropagation adds another term to the update value
$$
v_t = \eta \nabla_{\theta_{t-1}} J\left( \theta_{t-1} \right) + \lambda v_{t-1},
$$
with $\lambda$ representing a momentum, scaling the weight change amount -- this adjusts the weight gradient more forcefully, allowing it to escape false minema.

## Batch and online backpropagation
Another problem to face in training NN is how often the weights should be updated; gradients may be calculated on an element-by-element basis for the training set, or summed into batches, such that the weights are updated once per batch. We refer to these differences as

- Online training: update weights based on gradients from a *single training set element*
- Batch training: update weights based on sum gradient over *all training set elements*

For batch training, there is the question of batch size -- once can use anywhere from all of the elements to just pairs of the training set.

We introduce the terminology: *step/iteration* for the number of batches processed, and *epoch* for the number of times the training set has been processed.

## Stochastic gradient descent
[Stochastic gradient descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent) (SGD) is a batch training algorithm, with the exception that the batches are constructed from random sets of training elements, leading to irregular convergence. This is desired in the hope that it minimizes the chance of the network converging to a false minima.

SGD can be used with other concepts, such as momentum backpropagation, weight averaging, or e.g. adaptive gradient algorithms (AdaGrad).

## ADAM
[Adaptive Momentum Estimation](https://arxiv.org/abs/1412.6980) (ADAM) is an update to the RMS propagation algorithms; ADAM runs averages of both the gradients and the second moments of the gradients: that is to say it continously adjusts weights and momenta.

The first *moment* is the *mean*, and the second is the *variance*. We denote momentum now $\lambda \rightarrow m$, for consistency with literature. The parameter update is given by for iteration $t$ for first moment
$$
m_w^{t+1} = \beta_1 m_w^t + (1 - \beta_1) \nabla_w J^t
$$
and second moment
$$
v_w^{t+1} = \beta_2 v_w^t + (1 - \beta_2) \left( \nabla_w L^t \right) ^2
$$
To correct for the bias towards zero in the inital training cycles, we correct the moments with
$$
\hat{m}_w = \frac{m_w^{t+1}}{1- \beta_1^{t+1}}
$$
and
$$
\hat{v}_w = \frac{v_w^{t+1}}{1-\beta_2^{t+1}}
$$

The total update is then given by
$$
w^{t+1} = w^{t} - \eta \left( \frac{\hat{m}_w}{\sqrt{\hat{v}_w} + \epsilon} \right)
$$

We use a small value for $\epsilon \sim 10^{-8}$, and Kingma and Ba (2014) propose hyper parameters $\beta_1 = 0.9$ and $\beta_2 = 0.999$.

*Note:* squaring and square-rooting is done element wise.

## Application to TensorFlow
Tensorflow accepts the following update rules
- Adam
- AdaGrad
- [FTRL](https://www.eecs.tufts.edu/~dsculley/papers/ad-click-prediction.pdf)
- Momentum
- RMSProp
- SGD

We can specify them in our model during the model compilation stage: e.g.

In [ ]:
model.compile(
    optimizer='adam'
)